In [4]:
import pandas as pd
import os

def compute_RSI(series, window):
    delta = series.diff(1)
    gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
    RS = gain / loss
    RSI = 100 - (100 / (1 + RS))
    return RSI

name_stock = 'VN30F1M'
dir = r'C:\Users\PC\Dropbox\Projects\DataExport'

file = os.path.join(dir, name_stock + '.csv')

# Load data
data = pd.read_csv( file, parse_dates=['date'], index_col='date')

# Create technical indicators
data['SMA_20'] = data['close'].rolling(window=20).mean()
data['SMA_50'] = data['close'].rolling(window=50).mean()
data['RSI'] = compute_RSI(data['close'], window=14)  # define RSI computation separately


C:\Users\PC\AppData\Local\Temp\ipykernel_121656\2994746281.py:18: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv( file, parse_dates=['date'], index_col='date')


In [6]:
# Generate labels
data['Price_Diff'] = data['close'].shift(-5) - data['close']
data['Signal'] = 0
data.loc[data['Price_Diff'] > 0, 'Signal'] = 1  # 1 for buy
data.loc[data['Price_Diff'] < 0, 'Signal'] = -1  # -1 for sell

# Drop NaNs created by shifts
data = data.dropna()


In [7]:
from sklearn.model_selection import train_test_split

X = data[['SMA_20', 'SMA_50', 'RSI']]
y = data['Signal']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)


In [8]:
from sklearn.ensemble import RandomForestClassifier

# Initialize and train model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)


In [10]:
from zipline.api import order_target, record, symbol
from zipline import run_algorithm
import pandas as pd
from datetime import datetime
import pytz

# Define a function that trades based on the model predictions
def initialize(context):
    context.asset = symbol(name_stock)

def handle_data(context, data):
    # Get latest indicators
    current_data = pd.DataFrame({
        'SMA_20': data.history(context.asset, 'close', 20, '1d').mean(),
        'SMA_50': data.history(context.asset, 'close', 50, '1d').mean(),
        'RSI': compute_RSI(data.history(context.asset, 'close', 14, '1d'))
    }, index=[0])

    # Make a prediction
    signal = model.predict(current_data)[0]

    # Place orders based on signal
    if signal == 1:  # Buy
        order_target(context.asset, 100)
    elif signal == -1:  # Sell
        order_target(context.asset, -100)

# Define start and end dates for backtesting
start = datetime(2015, 1, 1, tzinfo=pytz.UTC)
end = datetime(2020, 1, 1, tzinfo=pytz.UTC)

# Run the backtest
results = run_algorithm(start=start, end=end, initialize=initialize, handle_data=handle_data,
                        capital_base=10000, data_frequency='daily', bundle='quantopian-quandl')


ValueError: no data for bundle 'quantopian-quandl' on or before 2024-11-09 02:16:36.058621+00:00
maybe you need to run: $ zipline ingest -b quantopian-quandl